In [5]:
import pymongo
import pandas as pd
import numpy as np
import math
from datetime import datetime

from sklearn.linear_model import LinearRegression ## likes, dislikes 대체

# import dask.dataframe as dd ## 대규모 데이터 처리
# import vaex ## 대규모 데이터 처리

import os
from dotenv import load_dotenv

In [6]:
file_path = 'C:/py_src/awake/data/'

In [7]:
# 계정 / 콘텐츠 분석 데이터셋 불러오기
merge_df_users_fin = pd.read_csv('C:/py_src/awake/data/merge_df_users_fin.csv', low_memory=False)
youtube_videos = pd.read_csv(file_path + 'youtube_videos.csv')

## 계정 데이터 분석

### 기초통계

In [8]:
# 기초통계
merge_df_users_fin[merge_df_users_fin.columns[11:-16]].describe()

,views,redViews,UNSUBSCRIBED,SUBSCRIBED,comments,likes,shares,dislikes,estimatedMinutesWatched,averageViewDuration,averageViewPercentage,estimated_revenue,estimated_ad_revenue,estimated_red_partner_revenue,gross_revenue,cpm,playback_based_cpm,subscribers_count,subscribers_gained,subscribers_lost
count,1.063740e+05,1.063740e+05,1.063740e+05,1.063740e+05,106374.000000,1.063740e+05,106374.000000,106374.000000,1.063740e+05,106374.000000,106374.000000,1.063740e+05,1.063740e+05,106374.000000,1.063740e+05,106374.000000,106374.000000,1.063740e+05,106374.000000,106374.000000
mean,7.619882e+04,1.174085e+04,5.742692e+04,8.736935e+03,36.363867,3.942476e+03,121.381964,38.403021,8.627140e+04,2.574108,34.820014,2.858782e+04,1.491898e+04,3227.336241,2.704715e+04,4998.146477,5463.550288,6.942734e+04,125.083723,41.489415
std,9.143264e+05,6.213386e+04,7.951268e+05,1.186929e+05,325.646665,7.598965e+04,1481.322268,460.897659,8.051993e+05,6.489120,55.104980,1.314624e+05,6.760697e+04,13108.338162,1.228179e+05,22819.456270,6549.813161,1.784480e+05,1274.779681,347.082501
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,-1.755000e+03,0.000000,-426.000000,0.000000e+00,0.000000,0.000000,-1.127185e+06,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000
25%,3.190000e+02,8.300000e+01,8.600000e+01,3.000000e+00,0.000000,2.000000e+00,1.000000,0.000000,5.220000e+02,0.780000,16.370000,1.074857e+02,4.233425e+01,7.570500,7.651000e+01,21.000000,3156.748750,3.509000e+03,1.000000,1.000000
50%,1.798000e+03,4.800000e+02,8.800000e+02,7.300000e+01,1.000000,1.900000e+01,6.000000,0.000000,2.637000e+03,1.570000,27.852500,1.446789e+03,8.109770e+02,129.617000,1.467412e+03,321.000000,5134.879500,1.485450e+04,8.000000,5.000000
75%,1.192850e+04,3.351000e+03,6.540250e+03,8.180000e+02,9.000000,1.590000e+02,33.000000,5.000000,2.012325e+04,2.580000,47.406000,1.091767e+04,5.008854e+03,1001.961000,9.066497e+03,2090.000000,7211.545500,5.016675e+04,43.000000,22.000000
max,4.064898e+07,1.862117e+06,3.707229e+07,4.546611e+06,35931.000000,3.054524e+06,64529.000000,24353.000000,3.330760e+07,120.230000,15049.760000,5.338431e+06,3.082096e+06,388432.785000,5.601879e+06,994860.000000,489556.625000,3.442657e+06,77446.000000,18821.000000


In [9]:
# 잘못된값 처리
merge_df_users_fin['estimated_revenue'] = np.where(merge_df_users_fin['estimated_revenue'] < 0,
                                                   merge_df_users_fin['estimated_ad_revenue'] + merge_df_users_fin['estimated_red_partner_revenue'],
                                                   merge_df_users_fin['estimated_revenue'])

### 상관관계

In [10]:
# 상관관계
users_corr_matrix = merge_df_users_fin[merge_df_users_fin.columns[11:-16]].corr()
users_corr_matrix

,views,redViews,UNSUBSCRIBED,SUBSCRIBED,comments,likes,shares,dislikes,estimatedMinutesWatched,averageViewDuration,averageViewPercentage,estimated_revenue,estimated_ad_revenue,estimated_red_partner_revenue,gross_revenue,cpm,playback_based_cpm,subscribers_count,subscribers_gained,subscribers_lost
views,1.000000,0.887217,0.990442,0.943632,0.513539,0.989550,0.977801,0.988344,0.977855,-0.018628,0.037962,0.707420,0.527650,0.564278,0.528820,0.512889,0.015630,0.476746,0.945027,0.957048
redViews,0.887217,1.000000,0.852688,0.829350,0.505960,0.830561,0.835933,0.870103,0.871826,-0.035783,0.070774,0.747019,0.569692,0.670986,0.570773,0.574830,0.008125,0.567490,0.807066,0.830839
UNSUBSCRIBED,0.990442,0.852688,1.000000,0.936748,0.490336,0.985803,0.972716,0.976796,0.967476,-0.016563,0.045201,0.677040,0.511669,0.537701,0.512846,0.498096,0.016228,0.457068,0.947833,0.953920
SUBSCRIBED,0.943632,0.829350,0.936748,1.000000,0.529486,0.950954,0.921657,0.925754,0.945686,-0.011343,0.037843,0.769068,0.591107,0.603208,0.592373,0.569447,0.029232,0.535271,0.840062,0.905883
comments,0.513539,0.505960,0.490336,0.529486,1.000000,0.500992,0.497737,0.524927,0.503917,-0.019272,0.044306,0.439505,0.325221,0.350877,0.325958,0.323260,0.008368,0.312219,0.504318,0.524517
likes,0.989550,0.830561,0.985803,0.950954,0.500992,1.000000,0.984210,0.977956,0.970681,-0.013316,0.027473,0.679996,0.505718,0.524619,0.506924,0.484652,0.018761,0.441218,0.936569,0.951122
shares,0.977801,0.835933,0.972716,0.921657,0.497737,0.984210,1.000000,0.967876,0.964019,-0.006713,0.028829,0.686033,0.529963,0.542920,0.530886,0.506686,0.025065,0.447365,0.938848,0.945530
dislikes,0.988344,0.870103,0.976796,0.925754,0.524927,0.977956,0.967876,1.000000,0.962861,-0.020187,0.037787,0.692616,0.500552,0.548058,0.501705,0.484625,0.013109,0.452624,0.958290,0.962210
estimatedMinutesWatched,0.977855,0.871826,0.967476,0.945686,0.503917,0.970681,0.964019,0.962861,1.000000,0.082759,0.026696,0.788172,0.657902,0.669712,0.658784,0.630563,0.034475,0.524246,0.912214,0.939517
averageViewDuration,-0.018628,-0.035783,-0.016563,-0.011343,-0.019272,-0.013316,-0.006713,-0.020187,0.082759,1.000000,-0.041275,0.105650,0.146831,0.365677,0.146073,0.099848,0.080594,0.031064,-0.015664,-0.015339


In [11]:
# 버그로 사용된 수치값 대체
merge_df_users_fin['likes'] = np.where(merge_df_users_fin['likes'] < 0, 0, merge_df_users_fin['likes'])
merge_df_users_fin['dislikes'] = np.where(merge_df_users_fin['dislikes'] < 0, 0, merge_df_users_fin['dislikes'])

### 파생변수

In [12]:
# 파생변수1 - 참여도 관련
merge_df_users_fin['like_rate'] = merge_df_users_fin['likes'] / merge_df_users_fin['views'] ## 좋아요 비율 
merge_df_users_fin['comment_rate'] = merge_df_users_fin['comments'] / merge_df_users_fin['views'] ## 댓글 비율
merge_df_users_fin['share_rate'] = merge_df_users_fin['shares'] / merge_df_users_fin['views'] ## 공유 비율  
merge_df_users_fin['dislike_rate'] = merge_df_users_fin['dislikes'] / merge_df_users_fin['views'] ## 싫어요 비율
merge_df_users_fin['total_engage_rate'] = (merge_df_users_fin['likes'] + merge_df_users_fin['comments'] + merge_df_users_fin['shares']) / merge_df_users_fin['views'] ## 총 참여율
merge_df_users_fin['comment_to_like_rate'] = merge_df_users_fin['comments'] / merge_df_users_fin['likes'] ## 댓글/좋아요 비율

In [13]:
# 파생변수2 - 구독자 관련
merge_df_users_fin['subscriber_increase_rate'] = merge_df_users_fin['subscribers_gained'] / merge_df_users_fin['subscribers_count'] ## 구독자 증가율
merge_df_users_fin['subscriber_decrease_rate'] = merge_df_users_fin['subscribers_lost'] / merge_df_users_fin['subscribers_count'] ## 구독자 감소율
merge_df_users_fin['net_subscribers_change'] = merge_df_users_fin['subscribers_gained'] - merge_df_users_fin['subscribers_lost'] ## 구독자 순증가

In [14]:
# 파생변수3 - 수익 관련
merge_df_users_fin['revenue_per_view'] = merge_df_users_fin['estimated_revenue'] / merge_df_users_fin['views'] ## 조회수당 수익
merge_df_users_fin['ad_revenue_rate'] = merge_df_users_fin['estimated_ad_revenue'] / merge_df_users_fin['estimated_revenue'] ## 광고수익비율
merge_df_users_fin['red_revenue_rate'] = merge_df_users_fin['estimated_red_partner_revenue'] / merge_df_users_fin['estimated_revenue'] ## 프리미엄수익비율
merge_df_users_fin['cpm_to_revenue_ratio'] = merge_df_users_fin['cpm'] - merge_df_users_fin['estimated_revenue'] ## cpm 대비 수익
merge_df_users_fin['revenue_per_ad_impression'] = merge_df_users_fin['estimated_revenue'] / merge_df_users_fin['ad_impressions'] ## 광고노출당 수익
merge_df_users_fin['playback_based_cpm_rate'] = merge_df_users_fin['playback_based_cpm'] / merge_df_users_fin['cpm'] ## 재생 기반 수익

In [15]:
# 파생변수4 - 시청 시간 관련
merge_df_users_fin['revenue_per_minute_watched'] = merge_df_users_fin['estimated_revenue'] / merge_df_users_fin['estimatedMinutesWatched'] ## 시청 시간 당 수익
merge_df_users_fin['avg_view_duration_rate'] = merge_df_users_fin['averageViewDuration'] / merge_df_users_fin['averageViewPercentage'] ## 평균 시청 시간 비율
merge_df_users_fin['watched_time_rate'] = merge_df_users_fin['averageViewPercentage'] * merge_df_users_fin['estimatedMinutesWatched'] ## 재생 비율 대비 시청 시간

In [16]:
# 파생변수5 - 구독자 유형 관련
merge_df_users_fin['subscribed_view_time_rate'] = merge_df_users_fin['SUBSCRIBED'] / (merge_df_users_fin['SUBSCRIBED'] + merge_df_users_fin['UNSUBSCRIBED']) ## 구독자 시청 시간 비율
merge_df_users_fin['unsubscribed_view_time_rate'] = merge_df_users_fin['UNSUBSCRIBED'] / (merge_df_users_fin['SUBSCRIBED'] + merge_df_users_fin['UNSUBSCRIBED']) ## 비구독자 시청 시간 비율

In [17]:
# 파생변수6 - 연령 및 성별 관련
female_col = [col_nm for col_nm in merge_df_users_fin.columns if 'female' in col_nm ]
male_col = [col_nm for col_nm in merge_df_users_fin.columns if ('male' in col_nm) and ('female' not in col_nm)]

youth_col = [col_nm for col_nm in merge_df_users_fin.columns if '13-17' in col_nm]
adult_col = [col_nm for col_nm in merge_df_users_fin.columns if ('18-24' in col_nm) or ('25-34' in col_nm) or ('35-44' in col_nm) or ('45-54' in col_nm)]
older_col = [col_nm for col_nm in merge_df_users_fin.columns if ('55-64' in col_nm) or ('65' in col_nm)]

merge_df_users_fin['female_viewer_rate'] = merge_df_users_fin[female_col].sum(axis=1) ## 여성 시청자 비율
merge_df_users_fin['male_viewer_rate'] = merge_df_users_fin[male_col].sum(axis=1) ## 남성 시청자 비율

merge_df_users_fin['youth_viewer_rate'] = merge_df_users_fin[youth_col].sum(axis=1) ## 청소년 시청자 비율
merge_df_users_fin['adult_viewer_rate'] = merge_df_users_fin[adult_col].sum(axis=1) ## 성인 시청자 비율
merge_df_users_fin['older_viewer_rate'] = merge_df_users_fin[older_col].sum(axis=1) ## 노인 시청자 비율

### y값 설정

#### 참여도 관련 지표 - 총 참여율

In [18]:
# 모두 0인 행 0으로 대체
merge_df_users_fin['total_engage_rate'] = merge_df_users_fin['total_engage_rate'].fillna(0)

In [19]:
merge_df_users_fin['total_engage_rate']

0         0.004723
1         0.006195
2         0.012912
3         0.009255
4         0.011152
            ...   
106369    0.000000
106370    0.000000
106371    0.000000
106372    0.000000
106373    0.000000
Name: total_engage_rate, Length: 106374, dtype: float64

#### 구독자 관련 지표 - 구독자 순증가

In [20]:
merge_df_users_fin['net_subscribers_change'].isnull().sum()

np.int64(0)

#### 시청 시간 관련 지표 - 시청유지율

In [21]:
merge_df_users_fin['averageViewPercentage'].isnull().sum()

np.int64(0)

#### 수익 관련 지표 - 조회수당 예상수익

In [22]:
# 모두 0인 행 0으로 대체
merge_df_users_fin['revenue_per_view'] = merge_df_users_fin['revenue_per_view'].fillna(0)

In [23]:
merge_df_users_fin['revenue_per_view']

0         2.168487
1         2.106251
2         2.119750
3         1.857470
4         2.126285
            ...   
106369    0.000000
106370    0.000000
106371    0.000000
106372    0.000000
106373    0.000000
Name: revenue_per_view, Length: 106374, dtype: float64

### 시계열 특성 파악

In [24]:
merge_df_users_fin.columns

Index(['youtube_user_id', 'date', 'channel_id', 'channel_title', 'phone_num',
       'report_user_id', 'published_at', 'viewCount', 'subscriberCount',
       'videoCount', 'yt_search_keyword', 'views', 'redViews', 'UNSUBSCRIBED',
       'SUBSCRIBED', 'comments', 'likes', 'shares', 'dislikes',
       'estimatedMinutesWatched', 'averageViewDuration',
       'averageViewPercentage', 'estimated_revenue', 'estimated_ad_revenue',
       'estimated_red_partner_revenue', 'gross_revenue', 'cpm',
       'playback_based_cpm', 'subscribers_count', 'subscribers_gained',
       'subscribers_lost', 'ad_impressions', 'monetized_playbacks',
       'age13-17.female', 'age13-17.male', 'age18-24.female', 'age18-24.male',
       'age25-34.female', 'age25-34.male', 'age35-44.female', 'age35-44.male',
       'age45-54.female', 'age45-54.male', 'age55-64.female', 'age55-64.male',
       'age65-.female', 'age65-.male', 'like_rate', 'comment_rate',
       'share_rate', 'dislike_rate', 'total_engage_rate',
     

In [33]:
merge_df_users_looker = merge_df_users_fin[['youtube_user_id','date','viewCount','subscriberCount','videoCount',
                                            'views','estimated_revenue','subscribers_count',
                                            'estimatedMinutesWatched','averageViewDuration',
                                            'total_engage_rate','net_subscribers_change','averageViewPercentage','revenue_per_view',
                                            'female_viewer_rate', 'male_viewer_rate', 'youth_viewer_rate', 'adult_viewer_rate', 'older_viewer_rate']]

In [34]:
merge_df_users_looker.to_csv('C:/py_src/awake/data/merge_df_users_looker.csv', encoding='utf-8-sig', index=False)

## 콘텐츠 분석 데이터셋

In [11]:
youtube_videos

,youtube_user_id,video,end_date,views,redViews,comments,likes,dislikes,shares,estimatedMinutesWatched,...,subscribersGained,subscribersLost,monetizedPlaybacks,adImpressions,cardClickRate,cardTeaserClickRate,cardImpressions,cardTeaserImpressions,cardClicks,cardTeaserClicks
0,64467ea09634a10e3709e1ea,--0HSDH6J7o,2023-04-23,3,1,0,0,0,0,0,...,0,0,0.0,0.0,0.0,0.0,0,0,0,0
1,64467ea09634a10e3709e1ea,--0HSDH6J7o,2023-04-30,4,0,0,0,0,0,1,...,0,0,0.0,0.0,0.0,0.0,0,0,0,0
2,64467ea09634a10e3709e1ea,--0HSDH6J7o,2023-05-01,2,1,0,0,0,0,0,...,0,0,0.0,0.0,0.0,0.0,0,0,0,0
3,64467ea09634a10e3709e1ea,--0HSDH6J7o,2023-05-02,3,0,0,0,0,0,0,...,0,0,0.0,0.0,0.0,0.0,0,0,0,0
4,64467ea09634a10e3709e1ea,--0HSDH6J7o,2023-05-03,1,0,0,0,0,0,0,...,0,0,0.0,0.0,0.0,0.0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8569506,63fb5daa2a0144119186eca8,NaN,2024-01-28,0,0,0,0,0,0,0,...,0,0,0.0,0.0,0.0,0.0,0,0,0,0
8569507,64809e9719c22b644dde6c44,NaN,2024-01-28,0,0,0,0,0,0,0,...,0,0,0.0,0.0,0.0,0.0,0,0,0,0
8569508,64bbf3cd616bd20e30379bf3,NaN,2024-01-28,0,0,0,0,0,0,0,...,0,0,0.0,0.0,0.0,0.0,0,0,0,0
8569509,6508ff021120b40b4427a4fc,NaN,2024-01-28,0,0,0,0,0,0,0,...,0,0,0.0,0.0,0.0,0.0,0,0,0,0
